In [12]:
!pip install --quiet azure-ai-projects==1.0.0b9 azure-identity azure-ai-evaluation dotenv

In [14]:
import dotenv
dotenv.load_dotenv()

True

In [15]:

import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet

# Import your custom functions to be used as Tools for the Agent
from user_functions import user_functions

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

AGENT_NAME = "Seattle Tourist Assistant"

# Add Tools to be used by Agent
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(toolset=toolset)

### Create an AI agent (Azure AI Agent Service)

In [4]:
agent = project_client.agents.create_agent(
    model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions="You are a helpful assistant",
    toolset=toolset,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_j72kXPo61Vhmnh3gKLURupIW


In [16]:
# Create a thread for the agent to interact with
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread

MESSAGE = "Can you email me weather info for Tokyo ?"

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

In [17]:
### Execute the Agent run
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

In [18]:
for message in project_client.agents.list_messages(thread.id, order="asc").data:
    print(f"Role: {message.role}")
    print(f"Content: {message.content[0].text.value}")
    print("-" * 40)

# Evaluate

### Get data from agent

In [19]:
from azure.ai.evaluation import AIAgentConverter
import json


# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

thread_id = thread.id
run_id = run.id
file_name = "evaluation_data.jsonl"

# Get a single agent run data
evaluation_data_single_run = converter.convert(thread_id=thread_id, run_id=run_id)

In [20]:
# Run this to save thread data to a JSONL file for evaluation
# Save the agent thread data to a JSONL file
evaluation_data = converter.prepare_evaluation_data(thread_ids=thread_id, filename=file_name)
print(json.dumps(evaluation_data, indent=4))

### Setting up evaluator

We will select the following evaluators to assess the different aspects relevant for agent quality: 

- [Intent resolution](https://aka.ms/intentresolution-sample): measures the extent of which an agent identifies the correct intent from a user query. Scale: integer 1-5. Higher is better.
- [Tool call accuracy](https://aka.ms/toolcallaccuracy-sample): evaluates the agent’s ability to select the appropriate tools, and process correct parameters from previous steps. Scale: float 0-1. Higher is better.
- [Task adherence](https://aka.ms/taskadherence-sample): measures the extent of which an agent’s final response adheres to the task based on its system message and a user query. Scale: integer 1-5. Higher is better.


In [10]:
from azure.ai.evaluation import (
    ToolCallAccuracyEvaluator,
    AzureOpenAIModelConfiguration,
    IntentResolutionEvaluator,
    TaskAdherenceEvaluator,
)
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)
# Needed to use content safety evaluators
azure_ai_project = {
    "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
    "project_name": os.environ["PROJECT_NAME"],
    "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
}

intent_resolution = IntentResolutionEvaluator(model_config=model_config)

tool_call_accuracy = ToolCallAccuracyEvaluator(model_config=model_config)

task_adherence = TaskAdherenceEvaluator(model_config=model_config)

### Run Evaluator

In [21]:
from azure.ai.evaluation import evaluate

response = evaluate(
    data=file_name,
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
        "intent_resolution": intent_resolution,
        "task_adherence": task_adherence,
    },
    azure_ai_project={
        "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
        "project_name": os.environ["PROJECT_NAME"],
        "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
    },
)
pprint(f'AI Foundary URL: {response.get("studio_url")}')

[2025-06-28 08:11:30 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-06-28 08:11:30 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-06-28 08:11:30 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_tool_call_accuracy_20250628_081130_861506, log path: /root/.promptflow/.runs/azure_ai_evaluation_evaluators_tool_call_accuracy_20250628_081130_861506/logs.txt
[2025-06-28 08:11:30 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_intent_resolution_20250628_081130_862089, log path: /root/.promptflow/.runs/azure_ai_evaluation_evaluators_intent_resolution_20250628_081130_862089/logs.txt
[2025-06-28 08:11:30 +0000][prompt

2025-06-28 08:11:35 +0000     717 execution.bulk     INFO     Finished 1 / 6 lines.
2025-06-28 08:11:35 +0000     717 execution.bulk     INFO     Average execution time for completed lines: 4.97 seconds. Estimated time for incomplete lines: 24.85 seconds.
2025-06-28 08:11:35 +0000     717 execution.bulk     INFO     Finished 2 / 6 lines.
2025-06-28 08:11:35 +0000     717 execution.bulk     INFO     Average execution time for completed lines: 2.51 seconds. Estimated time for incomplete lines: 10.04 seconds.
2025-06-28 08:11:36 +0000     717 execution.bulk     INFO     Finished 3 / 6 lines.
2025-06-28 08:11:36 +0000     717 execution.bulk     INFO     Average execution time for completed lines: 2.01 seconds. Estimated time for incomplete lines: 6.03 seconds.
2025-06-28 08:11:37 +0000     717 execution.bulk     INFO     Finished 4 / 6 lines.
2025-06-28 08:11:37 +0000     717 execution.bulk     INFO     Average execution time for completed lines: 1.54 seconds. Estimated time for incomplete